In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import itertools
from collections import Counter

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1057,tt0383010,30000000,54819301,The Three Stooges,Sean Hayes|Will Sasso|Chris Diamantopoulos|Sof...,Bobby Farrelly|Peter Farrelly,Just Say Moe.,While trying to save their childhood orphanage...,92,Comedy,Wessler Entertainment,4/13/2012,4.9,2012
1328,tt0397535,85000000,162242962,Memoirs of a Geisha,Zhang Ziyi|Gong Li|Youki Kudoh|Tsai Chin|Suzuk...,Rob Marshall,My world is as forbidden as it is fragile; wit...,A sweeping romantic epic set in Japan in the y...,145,Drama|History|Romance,DreamWorks SKG|Spyglass Entertainment|Columbia...,12/6/2005,7.1,2005
1377,tt0376105,30000000,93772522,Racing Stripes,Frankie Muniz|Mandy Moore|Hayden Panettiere|Br...,Frederik Du Chau,Cheer 'til you're horse!,Shattered illusions are hard to repair -- espe...,102,Comedy,Alcon Entertainment,1/6/2005,5.1,2005
1248,tt2017038,9000000,6108720,All Is Lost,Robert Redford,J.C. Chandor,Never give up.,"Deep into a solo voyage in the Indian Ocean, a...",106,Action|Adventure|Drama,Sudden Storm Productions|FilmNation Entertainm...,10/18/2013,6.5,2013
784,tt0479997,40000000,88100000,Season of the Witch,Nicolas Cage|Ron Perlman|Ulrich Thomsen|Christ...,Dominic Sena,Not all souls can be saved.,A 14th century Crusader (Cage) returns with hi...,95,Adventure|Fantasy|Action,Atlas Entertainment|Relativity Media,1/7/2011,5.2,2011


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [5]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...
def check_alpha(original_title):
    for symbol in original_title:
        count = 0
        if symbol.isalpha() or symbol.isdigit():
            count += 1
    return count

data['profit'] = data.apply(lambda x: x.revenue - x.budget, axis=1)
data['month'] = data.apply(lambda x: re.search(r'^[0-9]+', x.release_date).group(0), axis=1)
data["original_title"] = data["original_title"].astype(str)
data['film_lenth'] = data.apply(lambda x: check_alpha(x.original_title), axis=1)
data["overview"] = data["overview"].astype(str)
data['film_overview_lenth'] = data.apply(lambda x: len(x.overview), axis=1)

def get_genres(data):
    result = set()
    gen_uniq = data.genres.unique()
    for x in gen_uniq:
        gen = x.split('|')
        for c in gen:
            result.add(c)
    return result

def get_popular_genres(genres):
    rate = 0
    result = ''
    for genre in genres:
        current_genres = data[data['genres'].str.contains(genre, regex=False)].imdb_id.count()
        if current_genres > rate:
            rate = current_genres
            result = genre
    return result

def get_popular_genres_sum(genres):
    rate = 0
    result = ''
    for genre in genres:
        current_genres = data[data['genres'].str.contains(genre, regex=False)].profit.sum()
        if current_genres > rate:
            rate = current_genres
            result = genre
    return result

def get_flat_dataframe_by_production_companies(data):
    row_buffer = []
    for index, row in data.iterrows():
        production_companies_split = row.production_companies.split('|')
        if len(production_companies_split) > 1:
            for production_companies in production_companies_split:
                new_row = row.copy()
                new_row['production_companies'] = production_companies
                row_buffer.append(new_row)
        else:
            row_buffer.append(row)
    new_data = pd.DataFrame(row_buffer)
    new_data.reset_index(inplace=True, drop=True)
    
    return new_data

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [6]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [7]:
# тут пишем ваш код для решения данного вопроса:
data[data['budget'] == data['budget'].max()].original_title

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

ВАРИАНТ 2

In [8]:
# можно добавлять разные варианты решения
max_bud = data['budget'].max()
data.query('budget == @max_bud').original_title

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

# 2. Какой из фильмов самый длительный (в минутах)?

In [9]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '1157. Gods and Generals (tt0279111)'

In [10]:
data[data['runtime'] == data['runtime'].max()].original_title

1157    Gods and Generals
Name: original_title, dtype: object

# 3. Какой из фильмов самый короткий (в минутах)?





In [11]:
answers['3'] = '768. Winnie the Pooh (tt1449283)'
data[data['runtime'] == data['runtime'].min()].original_title

768    Winnie the Pooh
Name: original_title, dtype: object

# 4. Какова средняя длительность фильмов?


In [12]:
answers['4'] = '109.6585494970884'
data['runtime'].mean()

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [13]:
answers['5'] = '107.0'
data['runtime'].median()

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [14]:
# лучше код получения столбца profit вынести в Предобработку что в начале
answers['6'] = '239. Avatar (tt0499549)'
data[data['profit'] == data['profit'].max()].original_title

239    Avatar
Name: original_title, dtype: object

# 7. Какой фильм самый убыточный? 

In [15]:
answers['7'] = '1245. The Lone Ranger (tt1210819)'
data[data['profit'] == data['profit'].min()].original_title

1245    The Lone Ranger
Name: original_title, dtype: object

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [16]:
answers['8'] = '1478'
data.query('revenue > budget').imdb_id.count()

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [17]:
answers['9'] = '599. The Dark Knight (tt0468569)'
films_2008 = data.query('release_year == 2008')
films_2008[films_2008['revenue'] == films_2008['revenue'].max()].original_title

599    The Dark Knight
Name: original_title, dtype: object

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [18]:
answers['10'] = '1245. The Lone Ranger (tt1210819)'
films_2012_2014 = data.query('release_year >= 2012 & release_year <= 2014')
films_2012_2014[films_2012_2014['profit'] == films_2012_2014['profit'].min()].original_title

1245    The Lone Ranger
Name: original_title, dtype: object

# 11. Какого жанра фильмов больше всего?

In [19]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
answers['11'] = 'Drama'
list_of_genres = get_genres(data)
get_popular_genres(list_of_genres)

'Drama'

ВАРИАНТ 2

In [20]:
popular_film_counter = Counter()
data.apply(lambda row: popular_film_counter.update(row.genres.split('|')), axis=1)
popular_film_counter.most_common(1)

[('Drama', 782)]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [21]:
answers['12'] = 'Drama'
popular_genres_counter = Counter()
data[data['profit'] > 0].apply(lambda row: popular_genres_counter.update(row.genres.split('|')), axis=1)
popular_genres_counter.most_common(1)

[('Drama', 560)]

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [22]:
answers['13'] = 'Peter Jackson'
data.pivot_table(index = 'director', values = 'revenue', aggfunc = 'sum').sort_values(by=['revenue'], ascending=False).iloc[0]

revenue    6490593685
Name: Peter Jackson, dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [23]:
answers['14'] = 'Robert Rodriguez'
popular_director_counter = Counter()
data[data['genres'].str.contains('Action', regex=False)].apply(lambda row: popular_director_counter.update(row.director.split('|')), axis=1)
popular_director_counter.most_common(1)

[('Robert Rodriguez', 9)]

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [24]:
answers['15'] = 'Chris Hemsworth'
popular_director_counter = Counter()
data[(data.release_year == 2012) & (data.revenue == data[data.release_year == 2012].revenue.max())].apply(lambda row: popular_director_counter.update(row.cast.split('|')), axis=1)
popular_director_counter.most_common()

[('Robert Downey Jr.', 1),
 ('Chris Evans', 1),
 ('Mark Ruffalo', 1),
 ('Chris Hemsworth', 1),
 ('Scarlett Johansson', 1)]

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [25]:
answers['16'] = 'Matt Damon'
popular_cast_counter = Counter()
data[data.budget > data.budget.mean()].apply(lambda row: popular_cast_counter.update(row.cast.split('|')), axis=1)
popular_cast_counter.most_common(1)

[('Matt Damon', 18)]

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [26]:
answers['17'] = 'Action'
popular_director_counter = Counter()
data[data['cast'].str.contains('Nicolas Cage', regex=False)].apply(lambda row: popular_director_counter.update(row.genres.split('|')), axis=1)
popular_director_counter.most_common(1)

[('Action', 17)]

# 18. Самый убыточный фильм от Paramount Pictures

In [27]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'
data[data['production_companies'].str.contains('Paramount Pictures', regex=False)].sort_values(by=['profit']).head(1).original_title

925    K-19: The Widowmaker
Name: original_title, dtype: object

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [28]:
answers['19'] = '2015'
data.groupby(['release_year']).sum().sort_values(by=['revenue'], ascending=False).head(1)

,budget,revenue,runtime,vote_average,profit,film_lenth,film_overview_lenth
release_year,,,,,,,
2015,6780630004,25449202382,13355,755.2,18668572378,117,32445


# 20. Какой самый прибыльный год для студии Warner Bros?

In [29]:
answers['20'] = '2014'
data[data['production_companies'].str.contains('Warner Bros', regex=False)].groupby(['release_year']).sum().sort_values(by=['profit'], ascending=False).head(1)

,budget,revenue,runtime,vote_average,profit,film_lenth,film_overview_lenth
release_year,,,,,,,
2014,947600000,3243064519,1576,88.9,2295464519,13,4024


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [30]:
answers['21'] = 'Сентябрь'
data.groupby(['month']).count().sort_values(by=['imdb_id'], ascending=False).head(1)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,film_lenth,film_overview_lenth
month,,,,,,,,,,,,,,,,,
9,227,227,227,227,227,227,227,227,227,227,227,227,227,227,227,227,227


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [31]:
answers['22'] = '450'
data[data.release_date.str.contains(r'^[6\/, 7\/, 8\/]')].count().imdb_id

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [32]:
answers['23'] = 'Peter Jackson'
popular_director_counter = Counter()
data[data.release_date.str.contains(r'^(12\/|1\/|2\/)')].apply(lambda row: popular_director_counter.update(row.director.split('|')), axis=1)
popular_director_counter.most_common(1)

/workflow/programs/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


[('Peter Jackson', 7)]

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [33]:
answers['24'] = 'Universal Pictures (Universal)'
flat_dataframe_by_production_companies = get_flat_dataframe_by_production_companies(data)
flat_dataframe_by_production_companies.groupby(['production_companies']).sum().sort_values(by=['film_lenth'], ascending=False).head(1)

,budget,revenue,runtime,vote_average,release_year,profit,film_lenth,film_overview_lenth
production_companies,,,,,,,,
Universal Pictures,10644746652,31551297223,19475,1066.0,347292,20906550571,170,54493


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [34]:
answers['25'] = 'Midnight Picture Show'
flat_dataframe_by_production_companies = get_flat_dataframe_by_production_companies(data)
flat_dataframe_by_production_companies.groupby(['production_companies']).mean().sort_values(by=['film_overview_lenth'], ascending=False).head(1)

,budget,revenue,runtime,vote_average,release_year,profit,film_lenth,film_overview_lenth
production_companies,,,,,,,,
Midnight Picture Show,15000000.0,3533227.0,98.0,5.7,2008.0,-11466773.0,1.0,1000.0


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [35]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'
data[data['vote_average'] > np.percentile(data['vote_average'], 99)]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,film_lenth,film_overview_lenth
9,tt2096673,175000000,853708609,Inside Out,Amy Poehler|Phyllis Smith|Richard Kind|Bill Ha...,Pete Docter,Meet the little voices inside your head.,"Growing up can be a bumpy road, and it's no ex...",94,Comedy|Animation|Family,Walt Disney Pictures|Pixar Animation Studios|W...,6/9/2015,8.0,2015,678708609,6,1,638
34,tt3170832,6000000,35401758,Room,Brie Larson|Jacob Tremblay|Joan Allen|Sean Bri...,Lenny Abrahamson,Love knows no boundaries,Jack is a young boy of 5 years old who has liv...,117,Drama|Thriller,Element Pictures|No Trace Camping|A24|Duperele...,10/16/2015,8.0,2015,29401758,10,1,243
118,tt0816692,165000000,621752480,Interstellar,Matthew McConaughey|Jessica Chastain|Anne Hath...,Christopher Nolan,Mankind was born on Earth. It was never meant ...,Interstellar chronicles the adventures of a gr...,169,Adventure|Drama|Science Fiction,Paramount Pictures|Legendary Pictures|Warner B...,11/5/2014,8.0,2014,456752480,11,1,222
119,tt2015381,170000000,773312399,Guardians of the Galaxy,Chris Pratt|Zoe Saldana|Dave Bautista|Vin Dies...,James Gunn,All heroes start somewhere.,"Light years from Earth, 26 years after being a...",121,Action|Science Fiction|Adventure,Marvel Studios|Moving Picture Company (MPC)|Bu...,7/30/2014,7.9,2014,603312399,7,1,164
125,tt2084970,14000000,233555708,The Imitation Game,Benedict Cumberbatch|Keira Knightley|Matthew G...,Morten Tyldum,The true enigma was the man who cracked the code.,Based on the real life story of legendary cryp...,113,History|Drama|Thriller|War,Black Bear Pictures|Bristol Automotive,11/14/2014,8.0,2014,219555708,11,1,287
128,tt2267998,61000000,369330363,Gone Girl,Ben Affleck|Rosamund Pike|Carrie Coon|Neil Pat...,David Fincher,You don't know what you've got 'til it's...,With his wife's disappearance having become th...,145,Mystery|Thriller|Drama,Twentieth Century Fox Film Corporation|Regency...,10/1/2014,7.9,2014,308330363,10,1,169
138,tt2278388,30000000,174600318,The Grand Budapest Hotel,Ralph Fiennes|Tony Revolori|F. Murray Abraham|...,Wes Anderson,A perfect holiday without leaving home.,The Grand Budapest Hotel tells of a legendary ...,99,Comedy|Drama,Fox Searchlight Pictures|Scott Rudin Productio...,2/26/2014,7.9,2014,144600318,2,1,402
370,tt1375666,160000000,825500000,Inception,Leonardo DiCaprio|Joseph Gordon-Levitt|Ellen P...,Christopher Nolan,Your mind is the scene of the crime.,"Cobb, a skilled thief who commits corporate es...",148,Action|Thriller|Science Fiction|Mystery|Adventure,Legendary Pictures|Warner Bros.|Syncopy,7/14/2010,7.9,2010,665500000,7,1,280
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,7/16/2008,8.1,2008,816921825,7,1,396
872,tt0253474,35000000,120072577,The Pianist,Adrien Brody|Thomas Kretschmann|Frank Finlay|M...,Roman Polanski,Music was his passion. Survival was his master...,The Pianist is a film adapted from the biograp...,150,Drama|War,Bac Films|Canal+Polska|Heritage Films|Studio B...,9/24/2002,7.9,2002,85072577,9,1,238


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [36]:
answers['27'] = 'Daniel Radcliffe & Rupert Grint'
Counter(np.sum([list(itertools.combinations(x, 2)) for x in data['cast'].apply(lambda x: x.split('|'))])).most_common(1)

[(('Daniel Radcliffe', 'Rupert Grint'), 8)]

ВАРИАНТ 2

# Submission

In [37]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '723. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '1157. Gods and Generals (tt0279111)',
 '3': '768. Winnie the Pooh (tt1449283)',
 '4': '109.6585494970884',
 '5': '107.0',
 '6': '239. Avatar (tt0499549)',
 '7': '1245. The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': '599. The Dark Knight (tt0468569)',
 '10': '1245. The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'Сентябрь',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Universal Pictures (Universal)',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [38]:
# и убедиться что ни чего не пропустил)
len(answers)

27